In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, GPT2Config
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from tqdm import tqdm
import pandas as pd
import wandb

In [2]:
test_df = pd.read_csv('data/test.csv')

In [3]:
test_df.columns

Index(['review', 'sentiment'], dtype='object')

In [4]:
class MoviewReviewData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [5]:
#X_train = train_data['review']
#y_train = train_data['sentiment']
X_test = test_df['review']
y_test = test_df['sentiment']

In [6]:
model = GPT2ForSequenceClassification.from_pretrained('data/result/fine_tuned_gpt2_model2')
tokenizer = GPT2Tokenizer.from_pretrained('data/result/fine_tuned_gpt2_model2')

In [7]:
test_encodings = tokenizer(test_df['review'].tolist(), truncation=True, padding=True, max_length=512)

In [8]:
#y_train_numeric = y_train.map({'positive': 1, 'negative': 0}).astype(int)
y_test_numeric = y_test.map({'positive': 1, 'negative': 0}).astype(int)

In [9]:
from torch.utils.data import DataLoader

#test_encodings = tokenizer(test_data['review'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")

# Convert labels to a tensor
labels = torch.tensor(y_test_numeric)

# Create a DataLoader for the test set (optional, for batch processing)
test_dataset = MoviewReviewData(test_encodings, labels)
test_loader = DataLoader(test_dataset, batch_size=16)

def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    correct_predictions = 0
    total_predictions = 0

    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

# Evaluate the model
accuracy = evaluate(model, test_loader)
print(f"Model accuracy on the test set: {accuracy:.2f}%")

/tmp/ipykernel_3826431/442884725.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Model accuracy on the test set: 94.02%
